## *Import Libraries*

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

## *Load Cleaned Data*

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
%time df = pd.read_csv('clean_final.csv') # parse_dates = ['FL_DATE'])

Wall time: 31.2 s


In [5]:
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,AIR_TIME,DISTANCE,DIV_AIRPORT_LANDINGS
0,2019,1,1,3,4,2019-01-03,9E,20363,9E,N195PQ,5121,15412,35412,TYS,"Knoxville, TN",TN,47,Tennessee,54,10397,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,1140,1205.0,25.0,25.0,1.0,1.0,1100-1159,30.0,1311.0,4.0,1250,1.0,1.0,1200-1259,0.0,0.0,70.0,36.0,152.0,0
1,2019,1,1,4,5,2019-01-04,9E,20363,9E,N919XJ,5121,15412,35412,TYS,"Knoxville, TN",TN,47,Tennessee,54,10397,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,1140,1250.0,70.0,70.0,1.0,4.0,1100-1159,35.0,1403.0,9.0,1250,1.0,5.0,1200-1259,0.0,0.0,70.0,38.0,152.0,0
2,2019,1,1,5,6,2019-01-05,9E,20363,9E,N316PQ,5122,10397,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14783,34783,SGF,"Springfield, MO",MO,29,Missouri,64,950,956.0,6.0,6.0,0.0,0.0,0900-0959,20.0,1040.0,3.0,1051,0.0,-1.0,1000-1059,0.0,0.0,121.0,84.0,563.0,0
3,2019,1,1,6,7,2019-01-06,9E,20363,9E,N325PQ,5122,10397,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14783,34783,SGF,"Springfield, MO",MO,29,Missouri,64,950,945.0,-5.0,0.0,0.0,-1.0,0900-0959,16.0,1026.0,3.0,1053,0.0,-2.0,1000-1059,0.0,0.0,123.0,85.0,563.0,0
4,2019,1,1,7,1,2019-01-07,9E,20363,9E,N904XJ,5122,10397,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14783,34783,SGF,"Springfield, MO",MO,29,Missouri,64,950,947.0,-3.0,0.0,0.0,-1.0,0900-0959,25.0,1040.0,4.0,1053,0.0,-1.0,1000-1059,0.0,0.0,123.0,88.0,563.0,0


In [6]:
df.shape

(7421901, 47)

In [7]:
df['ORIGIN_CITY_NAME'] = df['ORIGIN_CITY_NAME'].str.split(',', n=1, expand = True)[0]
df['DEST_CITY_NAME'] = df['DEST_CITY_NAME'].str.split(',', n=1, expand = True)[0]

In [8]:
lst = []
for col1 in df.columns:
    for col2 in df.columns:
        if col1 != col2:
            if df[col1].equals(df[col2]) == True:
                lst.append([col1, col2])
lst

[['OP_UNIQUE_CARRIER', 'OP_CARRIER'], ['OP_CARRIER', 'OP_UNIQUE_CARRIER']]

*For predicting if a flight is delayed we cannot use future information like Arrival Data. So we delete the columns*

In [9]:
# 'WHEELS_OFF', 'ARR_TIME', 'ARR_DELAY', 'ARR_DELAY_NEW'
df.drop(['OP_CARRIER', 'OP_CARRIER_AIRLINE_ID', 
         'ORIGIN_AIRPORT_ID', 'ORIGIN', 'ORIGIN_STATE_ABR', 
         'DEP_TIME_BLK', 'TAXI_OUT', 'WHEELS_ON', 'TAXI_IN',
         'DEST_AIRPORT_ID', 'DEST', 'DEST_STATE_ABR', 
         'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK'], axis = 1, inplace = True)

In [10]:
df.shape

(7421901, 32)

In [11]:
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_CITY_MARKET_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_CITY_MARKET_ID,DEST_CITY_NAME,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,CRS_ARR_TIME,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,AIR_TIME,DISTANCE,DIV_AIRPORT_LANDINGS
0,2019,1,1,3,4,2019-01-03,9E,N195PQ,5121,35412,Knoxville,47,Tennessee,54,30397,Atlanta,13,Georgia,34,1140,1205.0,25.0,25.0,1.0,1.0,1250,0.0,0.0,70.0,36.0,152.0,0
1,2019,1,1,4,5,2019-01-04,9E,N919XJ,5121,35412,Knoxville,47,Tennessee,54,30397,Atlanta,13,Georgia,34,1140,1250.0,70.0,70.0,1.0,4.0,1250,0.0,0.0,70.0,38.0,152.0,0
2,2019,1,1,5,6,2019-01-05,9E,N316PQ,5122,30397,Atlanta,13,Georgia,34,34783,Springfield,29,Missouri,64,950,956.0,6.0,6.0,0.0,0.0,1051,0.0,0.0,121.0,84.0,563.0,0
3,2019,1,1,6,7,2019-01-06,9E,N325PQ,5122,30397,Atlanta,13,Georgia,34,34783,Springfield,29,Missouri,64,950,945.0,-5.0,0.0,0.0,-1.0,1053,0.0,0.0,123.0,85.0,563.0,0
4,2019,1,1,7,1,2019-01-07,9E,N904XJ,5122,30397,Atlanta,13,Georgia,34,34783,Springfield,29,Missouri,64,950,947.0,-3.0,0.0,0.0,-1.0,1053,0.0,0.0,123.0,88.0,563.0,0


*Extract the hour of day from CRS_DEP_TIME*

In [12]:
df['CRS_DEP_TIME_HR'] = df['CRS_DEP_TIME'].astype(int).astype(str).apply(lambda x: x.zfill(4))
df['CRS_DEP_TIME_HR'] = df['CRS_DEP_TIME_HR'].str[:2].astype(int)

In [13]:
df['CRS_DEP_TIME_HR'].unique()

array([11,  9, 13, 14, 21,  8, 10,  6, 16, 15, 18, 12, 17, 20, 19,  5,  7,
       22, 23,  0,  3,  2,  4,  1])

*FLIGHT_COUNT per day*

In [14]:
df = df.sort_values(by = ['FL_DATE', 'CRS_DEP_TIME_HR']).reset_index(drop = True)

In [15]:
df['FLIGHT_COUNT'] = df.dropna().groupby(['TAIL_NUM', 'FL_DATE']).cumcount()+1
df['FLIGHT_COUNT'].unique()

array([ 1., nan,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15.])

*Prior info Origin, Delay, Hour*

In [16]:
def get_prior_info(df, column, name, replace_null):
    
    df = df.sort_values(by = ['TAIL_NUM', 'FL_DATE']).reset_index(drop = True)
    df[name] = df.loc[df['FLIGHT_COUNT'].shift(-1)>1, column]
    df[name] = df[name].shift()
    df[name] = df[name].fillna(value=replace_null)
    return df

In [17]:
df = get_prior_info(df, 'ORIGIN_CITY_NAME', 'PRIOR_ORIGIN_CITY_NAME', replace_null='None')
df = get_prior_info(df, 'DEP_DEL15', 'PRIOR_DELAY', replace_null=0)
df = get_prior_info(df, 'CRS_DEP_TIME_HR', 'PRIOR_DEP_HR', replace_null=0)

In [18]:
df[(df['MONTH'] == 1) 
            & (df['DAY_OF_MONTH'] == 3)
            & (df['TAIL_NUM'] == '215NV')].sort_values('DEP_TIME')[['TAIL_NUM',
                                                                    'FL_DATE',
                                                                    'FLIGHT_COUNT',
                                                                    'ORIGIN_CITY_NAME',
                                                                    'PRIOR_ORIGIN_CITY_NAME',
                                                                    'DEP_DEL15',
                                                                    'PRIOR_DELAY',
                                                                    'CRS_DEP_TIME_HR',
                                                                    'PRIOR_DEP_HR']]

,TAIL_NUM,FL_DATE,FLIGHT_COUNT,ORIGIN_CITY_NAME,PRIOR_ORIGIN_CITY_NAME,DEP_DEL15,PRIOR_DELAY,CRS_DEP_TIME_HR,PRIOR_DEP_HR
7,215NV,2019-01-03,1.0,Asheville,None,0.0,0.0,8,0.0
8,215NV,2019-01-03,2.0,Fort Lauderdale,Asheville,0.0,0.0,10,8.0
9,215NV,2019-01-03,3.0,Asheville,Fort Lauderdale,0.0,0.0,13,10.0
10,215NV,2019-01-03,4.0,Baltimore,Asheville,0.0,0.0,15,13.0
11,215NV,2019-01-03,5.0,Savannah,Baltimore,0.0,0.0,18,15.0
12,215NV,2019-01-03,6.0,Baltimore,Savannah,0.0,0.0,20,18.0


In [19]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], format='%Y-%m-%d')

In [20]:
#'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
import re
def add_datepart(df, fldname): 
    fld = df[fldname]
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[n] = getattr(fld.dt,n.lower()).astype(int)
    df['Elapsed'] = (fld - fld.min()).dt.days
    df.drop(fldname, axis=1, inplace=True)

In [21]:
add_datepart(df, 'FL_DATE')

#### *Categorical Features*

In [22]:
label = LabelEncoder()

df['TAIL_NUM'] = label.fit_transform(df['TAIL_NUM'])
df['OP_UNIQUE_CARRIER'] = label.fit_transform(df['OP_UNIQUE_CARRIER'])
df['ORIGIN_CITY_NAME'] = label.fit_transform(df['ORIGIN_CITY_NAME'])
df['ORIGIN_STATE_NM'] = label.fit_transform(df['ORIGIN_STATE_NM'])
df['DEST_CITY_NAME'] = label.fit_transform(df['DEST_CITY_NAME'])
df['DEST_STATE_NM'] = label.fit_transform(df['DEST_STATE_NM'])
df['PRIOR_ORIGIN_CITY_NAME'] = label.fit_transform(df['PRIOR_ORIGIN_CITY_NAME'])

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7421901 entries, 0 to 7421900
Data columns (total 43 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   YEAR                    int64  
 1   QUARTER                 int64  
 2   MONTH                   int64  
 3   DAY_OF_MONTH            int64  
 4   DAY_OF_WEEK             int64  
 5   OP_UNIQUE_CARRIER       int32  
 6   TAIL_NUM                int32  
 7   OP_CARRIER_FL_NUM       int64  
 8   ORIGIN_CITY_MARKET_ID   int64  
 9   ORIGIN_CITY_NAME        int32  
 10  ORIGIN_STATE_FIPS       int64  
 11  ORIGIN_STATE_NM         int32  
 12  ORIGIN_WAC              int64  
 13  DEST_CITY_MARKET_ID     int64  
 14  DEST_CITY_NAME          int32  
 15  DEST_STATE_FIPS         int64  
 16  DEST_STATE_NM           int32  
 17  DEST_WAC                int64  
 18  CRS_DEP_TIME            int64  
 19  DEP_TIME                float64
 20  DEP_DELAY               float64
 21  DEP_DELAY_NEW           float64

In [24]:
df.shape

(7421901, 43)

In [25]:
df.to_csv('Pre_Processed.csv', index = False)